In [1]:
import os
import sys
import inspect
import torch
import torchvision
import random
import cv2
import numpy as np
from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm
import matplotlib.pyplot as plt

current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, parent_dir)

import face_lib.models as mlib
import face_lib.datasets as dlib
import face_lib.utils as utils

# from config import training_args

In [2]:
device = torch.device("cuda:0")

In [3]:
# checkpoint_path = "/gpfs/gpfs0/r.karimov/models/pfe/first_ms1m_pfe/sota.pt"
checkpoint_path = "/trinity/home/r.kail/Faces/face-evaluation/exman/runs/000067-2021-06-07-12-20-37/checkpoints/sota.pth"
casia_dir = "/gpfs/gpfs0/r.karimov/casia"
cats_dir = "/gpfs/gpfs0/r.karimov/cats"

In [4]:
class Args:
    def __init__(self):
        self.train_file = os.path.join(casia_dir, "ldmarks.txt")
        self.casia_dir = os.path.join(casia_dir, "data_")
        self.cats_dir = os.path.join(cats_dir)
        self.try_times = 5
        self.is_debug = False
        self.in_size = (112, 112)
        self.num_face_pb = 4
        self.in_feats = 512


args = Args()

In [5]:
checkpoint = torch.load(checkpoint_path, map_location=device)

In [6]:
model = {
    "backbone": mlib.iresnet50(),
    "head": mlib.PFEHeadAdjustable(25088, args.in_feats),
}
model["backbone"].load_state_dict(checkpoint["backbone"])
model["head"].load_state_dict(checkpoint["head"])

model["backbone"] = model["backbone"].eval().to(device)
model["head"] = model["head"].eval().to(device)

In [7]:
cats_set = dlib.CatsDataset(args)
face_set = dlib.CASIAWebFace(args)

In [8]:
print("Cats : ", len(cats_set))
print("Faces : ", len(face_set))

Cats :  139958
Faces :  490871


In [9]:
slice_size = 10000
assert slice_size < min(len(cats_set), len(face_set))

cats_subset = torch.utils.data.Subset(cats_set, range(slice_size))
face_subset = torch.utils.data.Subset(face_set, range(slice_size))

In [10]:
def calculate_uncertainty(log_sig_sq):
    sq_sigmas = torch.exp(log_sig_sq)
    #     uncertainty = 1 / (1 / sq_sigmas).sum(axis=1)
    uncertainty = sq_sigmas.mean(axis=1)
    return uncertainty


@torch.no_grad()
def calculate_dataset_uncertainties(model, dataset, device=torch.device("cpu")):
    loader = DataLoader(dataset, batch_size=64, shuffle=False)

    uncertainties = []

    for batch, _ in tqdm(loader):
        batch = torch.tensor(batch, dtype=torch.float).to(device)

        feature, sig_feat = model["backbone"](batch)
        log_sig_sq = model["head"](sig_feat)

        uncertainty = calculate_uncertainty(log_sig_sq)
        uncertainties.append(uncertainty.cpu())

    return torch.cat(uncertainties, dim=0)

In [11]:
cats_uncertainties = calculate_dataset_uncertainties(model, cats_subset, device=device)

  0%|          | 0/157 [00:00<?, ?it/s]<ipython-input-10-be18dbc57f8c>:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch = torch.tensor(batch, dtype=torch.float).to(device)
100%|██████████| 157/157 [03:00<00:00,  1.15s/it]


In [12]:
face_uncertainties = calculate_dataset_uncertainties(model, face_subset, device=device)

  0%|          | 0/157 [00:00<?, ?it/s]<ipython-input-10-be18dbc57f8c>:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch = torch.tensor(batch, dtype=torch.float).to(device)
100%|██████████| 157/157 [00:16<00:00,  9.34it/s]


In [13]:
cats_scores = np.array(cats_uncertainties)
face_scores = np.array(face_uncertainties)

cats_targets = np.ones_like(cats_scores)
face_targets = np.zeros_like(face_scores)

scores = np.concatenate(
    (
        cats_scores,
        face_scores,
    ),
    axis=0,
)
targets = np.concatenate(
    (
        cats_targets,
        face_targets,
    ),
    axis=0,
)

In [14]:
from sklearn.metrics import roc_curve, auc

In [15]:
fpr, tpr, thresholds = roc_curve(targets, scores)
roc_auc = auc(fpr, tpr)

In [20]:
print("ROC_AUC : ", roc_auc)

fig, ax = plt.subplots(figsize=(8, 8))

ax.plot(fpr, tpr)
ax.plot((0, 1), (0, 1), c="grey", linestyle="dashed")
ax.set_xlabel("False Positive Rate")
ax.set_ylabel("True Positive Rate")
ax.set_title("ROC_AUC : " + str(roc_auc))
ax.set_aspect("equal")

plt.savefig("ROC_AUC", dpi=200)

ROC_AUC :  0.86344117
